# Raking in 2D

This notebook explains how to use the raking method on a small 2D example.

In [1]:
# Python modules
import altair as alt
import numpy as np
import pandas as pd
from scipy.linalg import lu_factor, lu_solve
from scipy.sparse.linalg import cg
from statistics import covariance

In [2]:
# Parameters
N = 10000 # Number of samples for Monte Carlo method

## Generating the data

We first begin by generating a balanced $3 \times 5$ matrix. We compute the corresponding margins and then we add noise to the data. This way we have margins that sum correctly (thus the raking problem indeed has a solution), but we avoid having observations that already match the margins before the raking. 

In [3]:
# Define variables and margins names
x1 = [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
x2 = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
margins = ['s1_1', 's1_2', 's1_3', 's1_4', 's1_5', 's2_1', 's2_2']

# Generate balanced table
rng = np.random.default_rng(0)
beta_ij = rng.uniform(low=2.0, high=3.0, size=(3, 5))

# Compute the margins
s1 = np.sum(beta_ij, axis=0)
s2 = np.sum(beta_ij, axis=1)

# Add noise to the data
mean = beta_ij.flatten(order='F') + rng.normal(0.0, 0.1, size=15)

We then generate samples for the observations in order to be able to compare the two methods of uncertainty propagation (Monte Carlo and combination of delta method and implicit function theorem).

In [4]:
# Define the covariance matrix of the observations
cov = 0.01 * np.ones((15, 15))
np.fill_diagonal(cov, np.arange(0.01, 0.16, 0.01))

# Generate samples
y = rng.multivariate_normal(mean, cov, N)

## Applying the raking procedure to the mean of the observations

We start by computing the constraint matrix and the margin vector.

In [5]:
def constraints_2D(s1, s2, I, J):
    A = np.zeros((J + I - 1, I * J))
    for j in range(0, J):
        for i in range(0, I - 1):
            A[J + i, j * I + i] = 1
            A[j, j * I + i] = 1
        A[j, j * I + I - 1] = 1
    s = np.concatenate([s1, s2[0:(I - 1)]])
    return (A, s)

In [6]:
(A, s) = constraints_2D(s1, s2, 3, 5)

We then compute the mean of the observations and rake the mean using the $\chi^2$ distance.

In [7]:
y_0 = np.mean(y, 0)

In [8]:
def raking_chi2_distance(y, q, A, s):
    M = np.shape(A)[0]
    N = np.shape(A)[1]
    y_hat = np.matmul(A, y)
    Phi = np.matmul(A, np.transpose(A * y * q))
    lambda_k = cg(Phi, y_hat - s)[0]
    beta = y * (1 - q * np.matmul(np.transpose(A), lambda_k))
    return (beta, lambda_k)

In [9]:
(beta_0, lambda_k) = raking_chi2_distance(y_0, np.ones(15), A, s)

We can verify that the solution of the raking problem verify the constraints:

$A \beta_0 = s$

In [10]:
np.allclose(np.matmul(A, beta_0), s)

True

We store the results into a dataframe for plotting.

In [11]:
df_raked = pd.DataFrame({'X1': x1, \
                         'X2': x2, \
                         'observations': y_0, \
                         'raked_values': beta_0})

## Propagate the uncertainties using Delta method and IFT

We start by computing the gradient of the raked values with respect to the observations $y$: $\frac{\partial \beta}{\partial y}$ and the gradient of the raked values with respect to the margins $s$: $\frac{\partial \beta}{\partial s}$ using the implicit function theorem.

In [12]:
def compute_gradient(beta_0, lambda_0, y, q, A):
    M = np.shape(A)[0]
    N = np.shape(A)[1]
    # Hessian of the distance function
    H1_beta = np.diag(1.0 / (q * y))
    H1_y = np.diag(- beta_0 / (q * np.square(y)))
    # Gradient with respect to beta and lambda
    H1_lambda = np.transpose(np.copy(A))
    H2_beta = np.copy(A)
    H2_lambda = np.zeros((M, M))    
    DH_beta_lambda = np.concatenate(( \
        np.concatenate((H1_beta, H1_lambda), axis=1), \
        np.concatenate((H2_beta, H2_lambda), axis=1)), axis=0)
    # Gradient with respect to y and s
    H1_s = np.zeros((N, M))
    H2_y = np.zeros((M, N))
    H2_s = - np.identity(M)    
    DH_y_s = np.concatenate(( \
        np.concatenate((H1_y, H1_s), axis=1), \
        np.concatenate((H2_y, H2_s), axis=1)), axis=0)
    # Solve the system
    Dh = np.zeros_like(DH_y_s)
    lu, piv = lu_factor(DH_beta_lambda)
    for i in range(0, N + M):
        Dh[:, i] = - lu_solve((lu, piv), DH_y_s[:, i])
    # Get the gradient
    dh_y = Dh[0:N, 0:N]
    dh_s = Dh[0:N, N:(N + M)]
    return (dh_y, dh_s)

In [13]:
(dh_y, dh_s) = compute_gradient(beta_0, lambda_k, y_0, np.ones(15), A)

We store the results into a dataframe for plotting.

In [14]:
df_y = []
df_s = []
for i in range(0, 15):
    df_y.append(pd.DataFrame({'raked_1': np.repeat(x1[i], 15), \
                              'raked_2': np.repeat(x2[i], 15), \
                              'X1': x1, \
                              'X2': x2, \
                              'grad_y': dh_y[i, :]}))
    df_s.append(pd.DataFrame({'raked_1': np.repeat(x1[i], 7), \
                              'raked_2': np.repeat(x2[i], 7), \
                              'margins': margins, \
                              'grad_s': dh_s[i, :]}))
df_y = pd.concat(df_y)
df_s = pd.concat(df_s)

Then we compute the covariance matrix of the raked values using the Delta method.

In [15]:
covariance_mean = np.matmul(dh_y, np.matmul(cov, np.transpose(dh_y)))

## Monte Carlo simulation

We first apply the raking procedure to each sample.

In [17]:
beta = np.zeros((N, 15))
for n in range(0, N):
    y_n = y[n, :]
    (beta_n, lambda_k) = raking_chi2_distance(y_n, np.ones(15), A, s)
    beta[n, :] = beta_n

We compute the mean of the raked values.

In [19]:
mean_draws = np.mean(beta, 0)

We compute the variance of the raked values.

In [20]:
covariance_draws = np.matmul(np.transpose(beta - mean_draws), beta - mean_draws) / (N - 1)

## Plots

In [21]:
initial = pd.DataFrame({'X1': x1, \
                         'X2': x2, \
                         'variance': np.arange(0.01, 0.16, 0.01)})
variance = pd.DataFrame({'X1': x1, \
                         'X2': x2, \
                         'variance': np.diag(covariance_mean)})
df_obs = df_raked.drop(columns=['raked_values']).rename(columns={'observations': 'Value'})
df_obs['Type'] = 'Initial'
df_obs['width'] = 1
df_obs = df_obs.merge(initial, how='inner', \
    left_on=['X1', 'X2'], \
    right_on=['X1', 'X2'])
df_raked = df_raked.drop(columns=['observations']).rename(columns={'raked_values': 'Value'})
df_raked['Type'] = 'Raked'
df_raked['width'] = 2
df_raked = df_raked.merge(variance, how='inner', \
    left_on=['X1', 'X2'], \
    right_on=['X1', 'X2'])
df_raked = pd.concat([df_obs, df_raked])
df_raked['Upper'] = df_raked['Value'] + np.sqrt(df_raked['variance'])
df_raked['Lower'] = df_raked['Value'] - np.sqrt(df_raked['variance'])

In [22]:
bar = alt.Chart(df_raked).mark_errorbar(clip=True, opacity=0.5).encode(
    alt.X('Upper:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title='Raked value')),
    alt.X2('Lower:Q'),
    alt.Y('X1:N', axis=alt.Axis(title='X1')),
    color=alt.Color('Type:N', legend=None),
    strokeWidth=alt.StrokeWidth('width:Q', legend=None)
)
point = alt.Chart(df_raked).mark_point(
    filled=True
).encode(
    alt.X('Value:Q'),
    alt.Y('X1:N'),
    color=alt.Color('Type:N'),
    shape=alt.Shape('Type:N')
)
chart = alt.layer(point, bar).resolve_scale(
    shape='independent',
    color='independent'
).facet(
    column=alt.Column('X2:N', header=alt.Header(title='X2', titleFontSize=24, labelFontSize=24)),
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
).configure_legend(
    labelFontSize=24,
    titleFontSize=24
)
chart

alt.FacetChart(...)

In [23]:
bar = alt.Chart(df_raked).mark_errorbar(clip=True, opacity=0.5).encode(
    alt.X('Upper:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title='Raked value')),
    alt.X2('Lower:Q'),
    alt.Y('X2:N', axis=alt.Axis(title='X2')),
    color=alt.Color('Type:N', legend=None),
    strokeWidth=alt.StrokeWidth('width:Q', legend=None)
)
point = alt.Chart(df_raked).mark_point(
    filled=True
).encode(
    alt.X('Value:Q'),
    alt.Y('X2:N'),
    color=alt.Color('Type:N'),
    shape=alt.Shape('Type:N')
)
chart = alt.layer(point, bar).resolve_scale(
    shape='independent',
    color='independent'
).facet(
    column=alt.Column('X1:N', header=alt.Header(title='X1', titleFontSize=24, labelFontSize=24)),
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
).configure_legend(
    labelFontSize=24,
    titleFontSize=24
)
chart

alt.FacetChart(...)

In [24]:
index = np.argmax(np.abs(df_y.grad_y))
index_var1 = df_y.iloc[index].X1
index_var2 = df_y.iloc[index].X2
df_y_loc = df_y.loc[(df_y.X1==index_var1)&(df_y.X2==index_var2)]
max_scale = max(abs(df_y_loc['grad_y'].min()), abs(df_y_loc['grad_y'].max()))

base = alt.Chart(df_y_loc).encode(
    x=alt.X('raked_1:N', axis=alt.Axis(title='X1')),
    y=alt.Y('raked_2:N', axis=alt.Axis(title='X2')),
)

heatmap = base.mark_rect().encode(
    color=alt.Color('grad_y:Q',
        scale=alt.Scale(scheme='redblue', domain=[-max_scale, max_scale]),
        legend=alt.Legend(title=['Effect of', 'one obs.']))
)

text = base.mark_text(baseline='middle').encode(
    alt.Text('grad_y:Q', format='.2f')
)

chart = alt.layer(heatmap, text
).properties(
    title='X1 = ' + str(int(index_var1)) + ' - X2 = ' + str(int(index_var2)),
    width=120,
    height=180
).configure_title(
    fontSize=12
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=10,
    titleFontSize=10
)
chart

alt.LayerChart(...)

In [25]:
index = np.argmax(np.abs(df_y.grad_y))
index_raked_1 = df_y.iloc[index].raked_1
index_raked_2 = df_y.iloc[index].raked_2
df_y_loc = df_y.loc[(df_y.raked_1==index_raked_1)&(df_y.raked_2==index_raked_2)]
max_scale = max(abs(df_y_loc['grad_y'].min()), abs(df_y_loc['grad_y'].max()))

base = alt.Chart(df_y_loc).encode(
    x=alt.X('X1:N', axis=alt.Axis(title='X1')),
    y=alt.Y('X2:N', axis=alt.Axis(title='X2')),
)

heatmap = base.mark_rect().encode(
    color=alt.Color('grad_y:Q',
        scale=alt.Scale(scheme='redblue', domain=[-max_scale, max_scale]),
        legend=alt.Legend(title=['Effect of', 'all obs.']))
)

text = base.mark_text(baseline='middle').encode(
    alt.Text('grad_y:Q', format='.2f')
)

chart = alt.layer(heatmap, text
).properties(
    title='X1 = ' + str(int(index_raked_1)) + ' - X2 = ' + str(int(index_raked_2)),
    width=120,
    height=180
).configure_title(
    fontSize=12
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=10,
    titleFontSize=10
)
chart

alt.LayerChart(...)

In [26]:
delta_method = pd.DataFrame({'X1': x1, \
                             'X2': x2, \
                             'mean': df_raked.loc[df_raked.Type=='Raked'].Value})
all_draws = pd.DataFrame({'X1': x1, \
                          'X2': x2, \
                          'all_draws': mean_draws})
df_both = delta_method.merge(all_draws, how='inner', \
    on=['X1', 'X2'])
min_x = min(df_both['mean'].min(), df_both['all_draws'].min())
max_x = max(df_both['mean'].max(), df_both['all_draws'].max())
chart = alt.Chart(df_both).mark_point(size=60).encode(
    x=alt.X('all_draws:Q', scale=alt.Scale(domain=[min_x, max_x], zero=False), axis=alt.Axis(title='Using all draws')),
    y=alt.Y('mean:Q', scale=alt.Scale(domain=[min_x, max_x], zero=False), axis=alt.Axis(title='Using the mean')),
    color=alt.Color('X1:N', legend=alt.Legend(title='X1')),
    shape=alt.Shape('X2:N', legend=alt.Legend(title='X2'))
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
)
chart

alt.Chart(...)

In [27]:
delta_method = pd.DataFrame({'X1': x1, \
                             'X2': x2, \
                             'delta_method': np.diag(covariance_mean)})
all_draws = pd.DataFrame({'X1': x1, \
                          'X2': x2, \
                          'all_draws': np.diag(covariance_draws)})
df_both = delta_method.merge(all_draws, how='inner', \
    left_on=['X1', 'X2'], \
    right_on=['X1', 'X2'])
min_x = min(df_both['delta_method'].min(), df_both['all_draws'].min())
max_x = max(df_both['delta_method'].max(), df_both['all_draws'].max())
chart = alt.Chart(df_both).mark_point(size=60).encode(
    x=alt.X('all_draws:Q', scale=alt.Scale(domain=[min_x, max_x], zero=False), axis=alt.Axis(title='Using all draws')),
    y=alt.Y('delta_method:Q', scale=alt.Scale(domain=[min_x, max_x], zero=False), axis=alt.Axis(title='Using delta method')),
    color=alt.Color('X1:N', legend=alt.Legend(title='X1')),
    shape=alt.Shape('X2:N', legend=alt.Legend(title='X2'))
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
)
chart

alt.Chart(...)